In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.lite as tflite
from tensorflow import keras
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.utils import img_to_array, load_img

2024-12-26 03:22:26.949303: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-26 03:22:26.949377: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-26 03:22:26.949417: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-26 03:22:26.960293: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Conversion

In [2]:
model_path = "xception_final_14_0.880.h5"
# model = keras.models.load_model(model_path, custom_objects={"CustomModel": CustomModel})
model = keras.models.load_model(
    model_path,
)

2024-12-26 03:22:45.826249: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-26 03:22:46.314117: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-26 03:22:46.319571: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
converter = tflite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("clothing-model.tflite", "wb") as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmps3k3xq0t/assets


INFO:tensorflow:Assets written to: /tmp/tmps3k3xq0t/assets
2024-12-26 03:23:15.551562: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-12-26 03:23:15.551619: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-12-26 03:23:15.552652: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmps3k3xq0t
2024-12-26 03:23:15.590200: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-12-26 03:23:15.590237: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmps3k3xq0t
2024-12-26 03:23:15.649960: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2024-12-26 03:23:15.675386: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-12-26 03:23:16.254295: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

Using the tflite model

In [4]:
# Preprocess a single image
def preprocess_image(image_path, input_size=299):
    """Preprocess an image for prediction."""
    img = load_img(image_path, target_size=(input_size, input_size))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)  # Preprocessing specific to Xception
    return img_array

In [11]:
test_image_path = (
    "../data/clothing-dataset-small/test/shoes/0dd87e47-ca85-4d5c-9fd1-59f5a01eb656.jpg"
)
input_size = 299
class_names = [
    "dress",
    "hat",
    "longsleeve",
    "outwear",
    "pants",
    "shirt",
    "shoes",
    "shorts",
    "skirt",
    "t-shirt",
]

In [12]:
interpreter = tflite.Interpreter(model_path="clothing-model.tflite")
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

In [13]:
preprocessed_image = preprocess_image(test_image_path, input_size)
interpreter.set_tensor(input_index, preprocessed_image)
interpreter.invoke()

In [14]:
preds = interpreter.get_tensor(output_index)
predicted_class = class_names[np.argmax(preds[0])]
scores = dict(zip(class_names, preds[0]))
print(f"Predicted class: {predicted_class}\nscores: {scores}")

Predicted class: shoes
scores: {'dress': -7.3566732, 'hat': -1.4685564, 'longsleeve': -4.7650948, 'outwear': -2.4155884, 'pants': 1.4425707, 'shirt': -4.2841487, 'shoes': 16.41865, 'shorts': -1.3817132, 'skirt': -4.5483966, 't-shirt': -1.494125}
